<a href="https://colab.research.google.com/github/fcoliveira-utfpr/agrometeorologia/blob/main/03_infos_clima_solo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Início - Bibliotecas**
---

In [ ]:
#importando bibliotecas
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
import requests
import os
import openpyxl
from google.colab import files
import matplotlib.pyplot as plt
!pip install --upgrade geobr   -q
import geobr
import geopandas as gpd
import folium
from geobr import read_municipality
import branca
from shapely.geometry import Point
import ipywidgets as widgets
from IPython.display import display

# **Ver ESTADOS e MUNICÍPIOS**
---

In [ ]:
# Escolha de estado e município
url_42 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/clima_solo_local.csv"
df42 = pd.read_csv(url_42)

estados = sorted(df42['Estado'].unique())

estado_dropdown = widgets.Dropdown(
    options=estados,
    description='Estado:',
    layout=widgets.Layout(width='250px')
)

municipio_dropdown = widgets.Dropdown(
    description='Município:',
    layout=widgets.Layout(width='350px')
)
def atualizar_municipios(change):
    estado_sel = change['new']
    municipios = sorted(
        df42[df42['Estado'] == estado_sel]['Município'].unique()
    )
    municipio_dropdown.options = municipios

estado_dropdown.observe(atualizar_municipios, names='value')

estado_dropdown.value = estados[0]

display(estado_dropdown, municipio_dropdown)

# Tabelas de informações por Município e Estado
---

In [ ]:
# ========================================================================
# Carrega Tabela de informações do clima, solo e latitude e longitude
# ========================================================================

municipio = "Dourados" ###### Nome aqui entre aspas

# ===================================================================
estado = "MS"   # AC, AL, AM, BA, CE, DF, ES, GO, MA, MG, MS, MT,
                # PA, PB, PE, PI, PR, RJ, RN, RO, RR, RS, SC, SE, SP, TO
# ===================================================================

# ========================================================================
url_42 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/clima_solo_local.csv"
df42 = pd.read_csv(url_42)

# Filtra município e estado
df41 = df42[(df42['Estado'] == estado) & (df42['Município'] == municipio) ].copy()

df40 = df41[['IBGE-Code', 'Município','Estado', 'Região','Altitude', 'DTA (mm/m)',
       'Köppen', 'latitude', 'longitude']]
df40

# **Visualizando municípios, latitude e longitude**
---

In [ ]:
# ============================================================
# Carrega os municípios de cada Estado
# ============================================================

# ===================================================================
estado = "SC"   # AC, AL, AM, BA, CE, DF, ES, GO, MA, MG, MS, MT,
                # PA, PB, PE, PI, PR, RJ, RN, RO, RR, RS, SC, SE, SP, TO
# ===================================================================

url_1 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/clima_solo_local.csv"
df1 = pd.read_csv(url_1)

# Filtra estado
df12 = df1[df1['Estado'] == estado].copy()

# ------------------------------------------------------------
# Carregar shapefile dos municípios do estado
# ------------------------------------------------------------
mun = read_municipality(code_muni=estado, year=2020)

# Garantir CRS geográfico
mun = mun.to_crs(epsg=4326)

# ------------------------------------------------------------
# Calcular latitude e longitude (centroide)
# ------------------------------------------------------------
mun["longitude"] = mun.geometry.centroid.x
mun["latitude"]  = mun.geometry.centroid.y

# ------------------------------------------------------------
# Criar mapa centrado no estado
# ------------------------------------------------------------
centro = mun.geometry.unary_union.centroid
mapa2 = folium.Map(location=[centro.y, centro.x], zoom_start=6)

# ------------------------------------------------------------
# Tooltip com nome + latitude + longitude
# ------------------------------------------------------------
tooltip = folium.GeoJsonTooltip(
    fields=["name_muni", "latitude", "longitude"],
    aliases=["Município:", "Latitude:", "Longitude:"],
    localize=True,
    sticky=True
)

# ------------------------------------------------------------
# Adicionar municípios ao mapa
# ------------------------------------------------------------
folium.GeoJson(
    mun,
    tooltip=tooltip,
    style_function=lambda x: {
        "fillColor": "#1f77b4",
        "color": "black",
        "weight": 0.8,
        "fillOpacity": 0.6,
    },
).add_to(mapa2)

mapa2

# **Visualizando municípios e DTA**
---

In [ ]:
# ============================================================
# Carrega os municípios de cada Estado
# ============================================================

# ===================================================================
estado = "PR"   # AC, AL, AM, BA, CE, DF, ES, GO, MA, MG, MS, MT,
                # PA, PB, PE, PI, PR, RJ, RN, RO, RR, RS, SC, SE, SP, TO
# ===================================================================

url_1 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/clima_solo_local.csv"
df1 = pd.read_csv(url_1)

# Filtra município e estado
df12 = df1[df1['Estado'] == estado].copy()

# Carregar shapefile do estado
mun = read_municipality(code_muni=estado, year=2020)

# Criar mapa centrado no estado
centro = mun.geometry.unary_union.centroid
mapa2 = folium.Map(location=[centro.y, centro.x], zoom_start=6)

# Merge
mun_merged = mun.merge(df12[['Município', 'DTA (mm/m)']],
                       left_on='name_muni',
                       right_on='Município',
                       how='left')

tooltip = folium.GeoJsonTooltip(
    fields=["name_muni", "DTA (mm/m)"],
    aliases=["Município:", "DTA (mm/m):"],
    sticky=True
)

folium.GeoJson(
    mun_merged,
    tooltip=tooltip,
    style_function=lambda x: {
        "fillColor": "#1f77b4",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.7,
    },
).add_to(mapa2)

mapa2


# **Visualizando municípios e clima Koppen-Geiger**
---

In [ ]:
# ============================================================
# Carrega os municípios de cada Estado
# ============================================================

# ===================================================================
estado = "RS"   # AC, AL, AM, BA, CE, DF, ES, GO, MA, MG, MS, MT,
                # PA, PB, PE, PI, PR, RJ, RN, RO, RR, RS, SC, SE, SP, TO
# ===================================================================

url_1 = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/clima_solo_local.csv"
df1 = pd.read_csv(url_1)

# Filtra município e estado
df13 = df1[df1['Estado'] == estado].copy()

# Padroniza nomes para evitar divergências
df13['Município'] = df13['Município'].str.upper()

# Carregar shapefile do estado
mun = read_municipality(code_muni=estado, year=2020)
mun['name_muni'] = mun['name_muni'].str.upper()

# Merge correto
mun_merged = mun.merge(
    df13[['Município', 'Köppen']],
    left_on='name_muni',
    right_on='Município',
    how='left'
)

# ============================================================
# Configuração da legenda e colormap
# ============================================================

classes = sorted(df13['Köppen'].dropna().unique())
n_classes = len(classes)

# colormap solicitado
cmap = branca.colormap.linear.Set1_09.scale(0, n_classes)

# cria dicionário: classe -> cor
color_dict = {classes[i]: cmap(i) for i in range(n_classes)}

# ============================================================
# Mapa folium
# ============================================================

centro = mun.geometry.unary_union.centroid
mapa = folium.Map(location=[centro.y, centro.x], zoom_start=6)

def style_function(feature):
    classe = feature['properties'].get('Köppen', None)
    color = color_dict.get(classe, "#cccccc")  # cinza para ausentes
    return {
        "fillColor": color,
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 0.7 if classe else 0.2,
    }

tooltip = folium.GeoJsonTooltip(
    fields=["name_muni", "Köppen"],
    aliases=["Município:", "Köppen:"],
    sticky=True
)

folium.GeoJson(
    mun_merged,
    style_function=style_function,
    tooltip=tooltip
).add_to(mapa)

mapa

# **Município e estado pela latitude e longitude**
---

In [ ]:
# ------------------------------------------
# 1. Coordenadas de entrada
# ------------------------------------------
lat = -29.68
lon = -51.13

# cria ponto como GeoDataFrame
ponto = gpd.GeoDataFrame(geometry=[Point(lon, lat)], crs="EPSG:4326")

# ------------------------------------------
# 2. Carregar municípios
# ------------------------------------------
mun = read_municipality(code_muni="all", year=2020)

# ------------------------------------------
# 3. Identificar município via join espacial
# ------------------------------------------
resultado = gpd.sjoin(ponto, mun, how="left", predicate="within")

# pega atributos relevantes
muni = resultado.iloc[0]

nome = muni['name_muni']
codigo = muni['code_muni']
estado = muni['abbrev_state']

# ------------------------------------------
# 4. Criar mapa no Folium
# ------------------------------------------
m = folium.Map(location=[lat, lon], zoom_start=10)

# adicionar o ponto com tooltip
folium.Marker(
    location=[lat, lon],
    tooltip=f"{nome} ({estado})\nCódigo: {codigo}"
).add_to(m)

# adicionar polígono do município
folium.GeoJson(
    mun[mun['name_muni'] == nome],
    tooltip=folium.GeoJsonTooltip(
        fields=['name_muni', 'code_muni', 'abbrev_state'],
        aliases=['Município:', 'Código IBGE:', 'UF:'],
        sticky=True
    ),
    style_function=lambda x: {
        "fillColor": "#1f77b4",
        "color": "black",
        "weight": 2,
        "fillOpacity": 0.2
    }
).add_to(m)

m